# TSP

Given a set of cities and distance between every pair of cities, the problem is to find the shortest possible route that visits every city exactly once and returns back to the starting point.

### Cities and distances

This problem is from Chapter 3, Section 1 (3.1 Prototype Example) of the Introduction to Operations Research,
7th edition by Hillier and Lieberman.

<div>
<img src="img_9.3-1_table.png" width="400"/>
</div>

<!-- ![title](img_9.3-1_network.png) -->
<div>
<img src="img_9.3-1_network.png" width="700"/>
</div>

reference:
https://ozgurakgun.github.io/ModRef2017/files/ModRef2017_MTSP.pdf

### Import packages

In [1]:
import pandas as pd
from docplex.mp.model import Model

### Intializing the data

In [2]:
# Initialize the problem data
df = pd.read_csv('data_arc.csv')
df

,origin,destination,OD_pair,distance
0,O,A,OA,40
1,O,C,OC,60
2,O,B,OB,50
3,A,B,AB,10
4,A,D,AD,70
5,B,C,BC,20
6,B,D,BD,55
7,B,E,BE,40
8,C,E,CE,50
9,D,E,DE,10


In [3]:
edges = list((t.origin, t.destination) for t in df.itertuples())
edges

[('O', 'A'),
 ('O', 'C'),
 ('O', 'B'),
 ('A', 'B'),
 ('A', 'D'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'E'),
 ('C', 'E'),
 ('D', 'E'),
 ('D', 'T'),
 ('E', 'T'),
 ('A', 'O'),
 ('C', 'O'),
 ('B', 'O'),
 ('B', 'A'),
 ('D', 'A'),
 ('C', 'B'),
 ('D', 'B'),
 ('E', 'B'),
 ('E', 'C'),
 ('E', 'D'),
 ('T', 'D'),
 ('T', 'E')]

In [4]:
cities = set(df['destination'])
cities

{'A', 'B', 'C', 'D', 'E', 'O', 'T'}

In [5]:
distance = dict([((t.origin, t.destination),t.distance ) for t in df.itertuples()])
distance

{('O', 'A'): 40,
 ('O', 'C'): 60,
 ('O', 'B'): 50,
 ('A', 'B'): 10,
 ('A', 'D'): 70,
 ('B', 'C'): 20,
 ('B', 'D'): 55,
 ('B', 'E'): 40,
 ('C', 'E'): 50,
 ('D', 'E'): 10,
 ('D', 'T'): 60,
 ('E', 'T'): 80,
 ('A', 'O'): 40,
 ('C', 'O'): 60,
 ('B', 'O'): 50,
 ('B', 'A'): 10,
 ('D', 'A'): 70,
 ('C', 'B'): 20,
 ('D', 'B'): 55,
 ('E', 'B'): 40,
 ('E', 'C'): 50,
 ('E', 'D'): 10,
 ('T', 'D'): 60,
 ('T', 'E'): 80}

### Create the model

In [6]:
m=Model('TSP')

#### <font color=green>  Decision Variables

Let $x_{ij}$ = 1 if edge ${ij}$ is used, 0 otherwise.
<br>
Let $x_{ij}$ $\epsilon$ $(0,1)$
<br> <br>

<br>
Let $u_{c}$ = the number of cities visited from the origin up to city $c$
<br>
$u_{c}$ $\epsilon$ ${\rm I\!R}$

In [7]:
x = m.binary_var_dict(edges, name = 'x')
u = m.continuous_var_dict(cities, name ='u')

#### <font color=green>  Objective Function

Minimize Z $$Z = \sum_{ij } x_{ij}d_{ij}$$

In [8]:
m.minimize(m.sum(distance[e]*x[e] for e in edges))

#### <font color=green>  Constraints

$$
\sum_{i,j \: \epsilon \:E} x_{i} = 1 
\qquad \forall  \enspace i \: \epsilon \: Cities
$$ 

$$
\sum_{j \: \epsilon \:E} x_{j} = 1 
\qquad \forall  \enspace j \: \epsilon \: Cities
$$ 

In [9]:
# Constraint 1: each city must be entered once
for c in cities:
    if c != 'T':
        m.add_constraint(m.sum(x[(i,j)] for i,j in edges if i==c)==1, ctname='city_out_'+ c)

# Constraint 2: each city must be exited once
for c in cities:
    if c != 'O':
        m.add_constraint(m.sum(x[(i,j)] for i,j in edges if j==c)==1, ctname='city_in_'+ c)

#### <font color=green>  Constraints cont.

$$
edges_i + 1 \geq edges_j + M(1-x_{ij})
$$ 

In [10]:
# Constraint 3: ensures that u_j = u_i + 1 if and only if x_ij = 1
for i,j in edges:
    if j!='O':
        m.add_indicator(x[(i,j)],u[(i)]+1==u[(j)], name='order_'+i+'_'+ j)

In [11]:
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: TSP

Minimize
 obj: 40 x_O_A + 60 x_O_C + 50 x_O_B + 10 x_A_B + 70 x_A_D + 20 x_B_C + 55 x_B_D
      + 40 x_B_E + 50 x_C_E + 10 x_D_E + 60 x_D_T + 80 x_E_T + 40 x_A_O
      + 60 x_C_O + 50 x_B_O + 10 x_B_A + 70 x_D_A + 20 x_C_B + 55 x_D_B
      + 40 x_E_B + 50 x_E_C + 10 x_E_D + 60 x_T_D + 80 x_T_E
Subject To
 city_out_O: x_O_A + x_O_C + x_O_B = 1
 city_out_B: x_B_C + x_B_D + x_B_E + x_B_O + x_B_A = 1
 city_out_A: x_A_B + x_A_D + x_A_O = 1
 city_out_C: x_C_E + x_C_O + x_C_B = 1
 city_out_D: x_D_E + x_D_T + x_D_A + x_D_B = 1
 city_out_E: x_E_T + x_E_B + x_E_C + x_E_D = 1
 city_in_T: x_D_T + x_E_T = 1
 city_in_B: x_O_B + x_A_B + x_C_B + x_D_B + x_E_B = 1
 city_in_A: x_O_A + x_B_A + x_D_A = 1
 city_in_C: x_O_C + x_B_C + x_E_C = 1
 city_in_D: x_A_D + x_B_D + x_E_D + x_T_D = 1
 city_in_E: x_B_E + x_C_E + x_D_E + x_T_E = 1
 order_O_A: x_O_A = 1 -> u_O - u_A = -1
 order_O_C: x_O_C = 1 -> u_O - u_C = -1
 order_O_B: 

In [12]:
m.parameters.timelimit=120
m.parameters.mip.strategy.branch=1
m.parameters.mip.tolerances.mipgap=0.15

solution = m.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_TimeLimit                               120
CPXPARAM_MIP_Tolerances_MIPGap                   0.14999999999999999
Tried aggregator 2 times.
MIP Presolve modified 9 coefficients.
Aggregator did 10 substitutions.
Reduced MIP has 23 rows, 42 columns, and 77 nonzeros.
Reduced MIP has 23 binaries, 0 generals, 0 SOSs, and 21 indicators.
Presolve time = 0.01 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 23 rows, 42 columns, and 77 nonzeros.
Reduced MIP has 23 binaries, 0 generals, 0 SOSs, and 21 indicators.
Presolve time = 0.01 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Clique table members: 23.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel 

In [13]:
m.get_solve_status()

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [14]:
solution.display()

solution for: TSP
objective: 190
x_O_A = 1
x_A_B = 1
x_B_C = 1
x_C_E = 1
x_D_T = 1
x_E_D = 1
u_T = 6.000
u_B = 2.000
u_A = 1.000
u_C = 3.000
u_D = 5.000
u_E = 4.000


In [15]:
lst = []
for i in x:
    if solution.get_var_value(x[i]) > 0:
        soln = (i[0], i[1], solution.get_var_value(x[i]))
        lst.append(soln)
df = pd.DataFrame.from_records(lst, columns=['starting city', 'destination city', 'solution'])
df

,starting city,destination city,solution
0,O,A,1.0
1,A,B,1.0
2,B,C,1.0
3,C,E,1.0
4,D,T,1.0
5,E,D,1.0


In [16]:
lst_x = []
for i in x:
    if solution.get_var_value(x[i]) > 0:
        soln_x = (i[0], i[1], solution.get_var_value(x[i]))
        lst_x.append(soln_x)
df_x = pd.DataFrame.from_records(lst_x, columns=['starting city', 'destination city', 'solution'])
df_x

,starting city,destination city,solution
0,O,A,1.0
1,A,B,1.0
2,B,C,1.0
3,C,E,1.0
4,D,T,1.0
5,E,D,1.0


In [17]:
lst_c = []
for c in u:
    soln_c = (c[0],solution.get_var_value(u[c]))
    lst_c.append(soln_c)
df_c = pd.DataFrame.from_records(lst_c, columns = ['city', 'visit order'])
df_c.sort_values(by=['visit order'], inplace = True)
df_c

,city,visit order
0,O,0.0
3,A,1.0
2,B,2.0
4,C,3.0
6,E,4.0
5,D,5.0
1,T,6.0


In [18]:
# Export results to csv
import os

base_dir = os.getcwd()

def export_soln_to_csv(df, model_name = 'untitled'):
    """ model refers to model object from docplex.mp.model"""

    try:
        os.mkdir(os.path.join(base_dir, 'output'))
    except:
        pass

    filename = 'output/' + 'soln_' + model_name + '.csv'
    solution_output = os.path.join(os.getcwd(), filename)
    df.to_csv(solution_output, index=False)

In [19]:
export_soln_to_csv(df_x, m.get_name()+'_arc')
export_soln_to_csv(df_c, m.get_name()+'_order')